In [1]:
import torch
import torchtext
from torchtext.datasets import Multi30k
from torchtext.vocab import build_vocab_from_iterator

import spacy

In [2]:
!python -m spacy download en_core_web_md
!python -m spacy download de_core_news_md
!pip install portalocker

2023-12-01 01:25:40.162676: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 01:25:40.162767: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 01:25:40.162812: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 01:25:41.739564: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 24.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
2023-12-01 01:25:57.998994: E tensorflow/compiler/xla/st

In [3]:
# load data sets
train_data, valid_data, test_data = Multi30k(language_pair= ('de', 'en'))
next(iter(train_data))

('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'Two young, White males are outside near many bushes.')

In [19]:
# define tokenizers
nlp_en = spacy.load("en_core_web_md")
nlp_de = spacy.load("de_core_news_md")

# create token iterables for each language
def get_en_tokens():
  for _, en_phrase in iter(train_data):
    yield [token.text for token in nlp_en.tokenizer(en_phrase)]

def get_de_tokens():
  for de_phrase, _ in iter(train_data):
    yield [token.text for token in nlp_en.tokenizer(de_phrase)]

tokens_en = get_en_tokens()
print(f"english tokens: {next(tokens_en)}")

# create english vocabulary
vocab_en = build_vocab_from_iterator(tokens_en, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

tokens_de = get_de_tokens()
print(f"deutch tokens: {next(tokens_de)}")

# create deutch vocabulary
vocab_de = build_vocab_from_iterator(tokens_de, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

# set unknown token index
vocab_de.set_default_index(vocab_de['<unk>'])
vocab_en.set_default_index(vocab_en['<unk>'])


/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/iter/combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


english tokens: ['Two', 'young', ',', 'White', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']
deutch tokens: ['Zwei', 'junge', 'weiße', 'Männer', 'sind', 'i', 'm', 'Freien', 'in', 'der', 'Nähe', 'vieler', 'Büsche', '.']


In [27]:
print(f"hello world I am being tokenized ->", end=" ")

[vocab_en[token.text] for token in nlp_en.tokenizer("hello world I am being tokenized")]

hello world I am being tokenized -> 

[5465, 1870, 1166, 3426, 194, 0]